   # SVD Matrix Factorization

In [5]:
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn.decomposition import TruncatedSVD

In [37]:
columns = ['userID','itemID','rating','timestamp']
frame= pd.read_csv('u.data', sep='\t' , names = columns)
frame['userID'].unique().size

943

In [7]:
movie_columns = ['itemID','movie title','release date','video release date','IMDb URL','unknown','Action','Adventure','Animation',
           'Childrens','Comedy','Crime','Documentary','Drama','Fantasy','Film-Noir','Horror',
           'Musical','Mystery','Romance','Sci-Fi','Thriller','War','Western']
movies=pd.read_csv('u.item',sep='|', names=movie_columns, encoding='latin-1')
movies_names=movies[['itemID','movie title']]

In [8]:
combined_movies_data= pd.merge(frame,movies_names,on='itemID')
combined_movies_data.head()

,userID,itemID,rating,timestamp,movie title
0,196,242,3,881250949,Kolya (1996)
1,63,242,3,875747190,Kolya (1996)
2,226,242,5,883888671,Kolya (1996)
3,154,242,3,879138235,Kolya (1996)
4,306,242,5,876503793,Kolya (1996)


In [9]:
combined_movies_data.groupby('itemID')['rating'].count().sort_values(ascending=False).head()

itemID
50     583
258    509
100    508
181    507
294    485
Name: rating, dtype: int64

In [11]:
Filter = combined_movies_data['itemID']==50
combined_movies_data[Filter]['movie title'].unique()

array(['Star Wars (1977)'], dtype=object)

In [13]:
rating_crosstab = combined_movies_data.pivot_table(values = 'rating',index = 'userID',columns = 'movie title',fill_value=0)
rating_crosstab.head()

movie title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,Á köldum klaka (Cold Fever) (1994)
userID,,,,,,,,,,,,,,,,,,,,,
1,0,0,2,5,0,0,3,4,0,0,...,0,0,0,5,3,0,0,0,4,0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,2,0,0,0,0,4,0,0,...,0,0,0,4,0,0,0,0,4,0


In [15]:
rating_crosstab.shape

(943, 1664)

## Transposing the matrix

In [38]:
X = rating_crosstab.values.T
X.shape

(1664, 943)

## Decomposing the matrix

In [26]:
SVD = TruncatedSVD (n_components=12,random_state=17)

resultant_matrix=SVD.fit_transform(X)

resultant_matrix.shape

(1664, 12)

### Generating a correlation matrix

In [42]:
corr_mat = np.corrcoef(resultant_matrix)
corr_mat.shape

(1664, 1664)

### Islotaing Star wars from Correlation matrix
Star wars is the movie of choice

In [52]:
movies_names= rating_crosstab.columns
movies_list=list(movies_names)
star_wars=movies_list.index('Star Wars (1977)')
star_wars

1398

In [55]:
corr_star_wars=corr_mat[star_wars]
corr_star_wars.shape

(1664,)

## Recommending a highly correlated movie with stars wars

In [63]:
list(movies_names[(corr_star_wars <1.0) & (corr_star_wars >0.9) ])

['Die Hard (1988)',
 'Empire Strikes Back, The (1980)',
 'Fugitive, The (1993)',
 'Raiders of the Lost Ark (1981)',
 'Return of the Jedi (1983)',
 'Terminator 2: Judgment Day (1991)',
 'Terminator, The (1984)',
 'Toy Story (1995)']

In [64]:
list(movies_names[(corr_star_wars <1.0) & (corr_star_wars >0.95) ])

['Return of the Jedi (1983)']